# Лабораторная работа №3

## Тензорные операции и их реализация в NumPy

### Цель работы
Изучение операций с тензорами произвольной размерности, включая:
- Тензорное транспонирование
- Тензорное умножение с заданными параметрами λ и μ
- Создание единичных тензоров

### Теоретическая часть

Тензорное умножение с параметрами λ и μ представляет собой обобщение матричного умножения на многомерные массивы. Параметры λ и μ определяют:
- λ - количество индексов для суммирования (контравариантные индексы)
- μ - количество индексов для внешнего произведения (ковариантные индексы)

Формально операция определяется как:
$$ C_{i_1...i_k,j_1...j_v} = \sum_{s_1...s_λ,c_1...c_μ} A_{i_1...i_k,s_1...s_λ,c_1...c_μ} \cdot B_{s_1...s_λ,c_1...c_μ,j_1...j_v} $$

In [ ]:
import numpy as np
import sympy as sp

# Инициализация параметров
p = 4
q = 4 
n = 2

# Генерация случайных тензоров
dims_A = np.random.randint(low=1, high=n+1, size=p)
dims_B = np.random.randint(low=1, high=n+1, size=q)
A = np.random.randint(-10, 10, size=dims_A)
B = np.random.randint(-10, 10, size=dims_B)

### Тензорное транспонирование

Транспонирование тензора - это перестановка его индексов согласно заданной перестановке:

In [ ]:
# Генерация перестановок для транспонирования
T_comb = np.array([3, 0, 1, 2])
A_T = np.random.choice(T_comb, p, replace=False)
B_T = np.random.choice(T_comb, q, replace=False)

# Применение транспонирования
A_transposed = A.transpose(A_T)
B_transposed = B.transpose(B_T)

### Реализация тензорного умножения

Функция mat_mult выполняет обобщенное тензорное умножение с параметрами λ и μ:

In [ ]:
def mat_mult(A: np.ndarray, B: np.ndarray, lambda_value: int, mu_value: int) -> np.ndarray:
    """
    Выполняет тензорное умножение с заданными параметрами λ и μ
    
    Параметры:
    A, B - входные тензоры
    lambda_value - количество индексов для суммирования
    mu_value - количество индексов для внешнего произведения
    
    Возвращает:
    Результирующий тензор
    """
    A_p = len(A.shape)
    B_q = len(B.shape)
    
    # Проверка корректности параметров
    if lambda_value < 0 or mu_value < 0:
        raise ValueError("Параметры lambda и mu должны быть неотрицательными")
        
    # Вычисление размерностей
    k = A_p - lambda_value - mu_value
    v = B_q - lambda_value - mu_value
    
    # Проверка совместимости размерностей
    if (A.shape[k:k+lambda_value] != B.shape[:lambda_value] or 
        A.shape[k+lambda_value:] != B.shape[lambda_value:lambda_value+mu_value]):
        raise ValueError("Несовместимые размерности тензоров")
    
    # Инициализация результата
    result_shape = A.shape[:k] + B.shape[lambda_value+mu_value:]
    D = np.zeros(result_shape if result_shape else (1,))
    
    # Вычисление произведения
    for l_idx in np.ndindex(A.shape[:k]):
        for m_idx in np.ndindex(B.shape[lambda_value+mu_value:]):
            sum_result = 0.0
            for s_idx in np.ndindex(A.shape[k:k+lambda_value]):
                for c_idx in np.ndindex(A.shape[k+lambda_value:]):
                    sum_result += A[l_idx + s_idx + c_idx] * B[s_idx + c_idx + m_idx]
            D[l_idx + m_idx] = sum_result
    
    return np.squeeze(D)

### Создание единичного тензора

Функция create_eye генерирует единичный тензор заданной размерности:

In [ ]:
def create_eye(lambda_value: int, mu_value: int, n: int) -> np.ndarray:
    """
    Создает единичный тензор с параметрами λ и μ
    
    Параметры:
    lambda_value - количество индексов для суммирования
    mu_value - количество индексов для внешнего произведения
    n - размерность каждого индекса
    
    Возвращает:
    Единичный тензор
    """
    if lambda_value < 0 or mu_value < 0 or n < 0:
        raise ValueError("Параметры должны быть неотрицательными")
    
    total_dims = 2 * (mu_value + lambda_value)
    shape = (n,) * total_dims
    E = np.zeros(shape, dtype=float)
    
    if total_dims == 0:
        return np.array(1.0)
    
    c_indices = (n,) * mu_value 
    m_indices = (n,) * mu_value 
    s_indices = (n,) * lambda_value 
    
    if mu_value == 0:
        for s_idx in np.ndindex(s_indices):  
            E[s_idx + s_idx] = 1.0  
    else:
        for c_idx in np.ndindex(c_indices):  
            for m_idx in np.ndindex(m_indices):  
                if c_idx == m_idx:  
                    for s_idx in np.ndindex(s_indices):
                        full_idx = c_idx + s_idx + m_idx
                        E[full_idx] = 1.0
    
    return E

### Пример использования

Демонстрация работы функций на конкретных примерах:

In [ ]:
# Пример тензорного умножения
D = mat_mult(A, B, lambda_value=0, mu_value=1)
print("Форма A:", A.shape)
print("Форма B:", B.shape)
print("Форма результата:", D.shape)

# Пример создания единичного тензора
E_l_m = create_eye(lambda_value=0, mu_value=1, n=2)
print("Единичный тензор:", E_l_m)

### Выводы

1. Реализованы базовые операции тензорной алгебры
2. Разработана универсальная функция тензорного умножения
3. Создана функция генерации единичных тензоров
4. Проведена проверка корректности работы на тестовых примерах